# WAVES

In [1]:
import pyodbc
import pandas as pd

In [2]:
cnxn = pyodbc.connect("Driver={SQL Server Native Client 11.0};"
                        "Server=wd02;"
                        "Database=FootPrint;"
                        "Trusted_Connection=yes;")

### Query

In [3]:
df = pd.read_sql_query("SELECT CONVERT(varchar,w.releaseDate,100) Date, w.id, wa.name Warehouse, s.lookupCode Shipments, p.name Customer, ss.name ShipmentStatus\
    FROM datex_footprint.Waves w\
    JOIN datex_footprint.WaveStatuses ws\
        ON ws.id = w.statusId\
    JOIN datex_footprint.Warehouses wa\
        ON wa.id = w.warehouseId\
    JOIN datex_footprint.shipments s\
        ON s.waveId = w.id\
    JOIN datex_footprint.ShipmentOrderLookup sol\
        ON sol.shipmentId = s.id\
    JOIN datex_footprint.Orders o\
        ON o.id = sol.orderId\
    JOIN datex_footprint.Projects p\
        ON p.id = o.projectId\
    JOIN datex_footprint.ShipmentStatuses ss\
        ON ss.id = s.statusId\
    WHERE releaseDate > dateadd(day,-1,getdate())\
        AND w.statusId = 3\
    ORDER BY releaseDate DESC", cnxn)

# Create Waves Table Data

In [4]:
df.head(10)

,Date,id,Warehouse,Shipments,Customer,ShipmentStatus
0,Aug 17 2022 10:37AM,244362,10,BPS200023,Beach Patient Assistance,Released
1,Aug 17 2022 10:22AM,244358,20,8394562,"ACI Healthcare USA, Inc.",Released
2,Aug 17 2022 10:21AM,244356,15,8409341,"Brookfield Pharmaceuticals, LLC",Released
3,Aug 17 2022 10:21AM,244356,15,8409342,"Brookfield Pharmaceuticals, LLC",Released
4,Aug 17 2022 10:21AM,244356,15,8409343,"Brookfield Pharmaceuticals, LLC",Released
5,Aug 17 2022 10:21AM,244356,15,8409344,"Brookfield Pharmaceuticals, LLC",Released
6,Aug 17 2022 10:21AM,244356,15,8409345,"Brookfield Pharmaceuticals, LLC",Released
7,Aug 17 2022 10:21AM,244356,15,8409346,"Brookfield Pharmaceuticals, LLC",Released
8,Aug 17 2022 10:21AM,244356,15,8409347,"Brookfield Pharmaceuticals, LLC",Released
9,Aug 17 2022 10:21AM,244356,15,8409348,"Brookfield Pharmaceuticals, LLC",Released


In [5]:
# Count Shipments per Wave and Status
count_shipments_statusdf = df.groupby(['id', 'ShipmentStatus']).count()[['Shipments']]
count_shipments_statusdf.reset_index(inplace=True)
count_shipments_statusdf.tail()

,id,ShipmentStatus,Shipments
35,244354,Released,4
36,244355,Released,25
37,244356,Released,23
38,244358,Released,1
39,244362,Released,1


In [6]:
# Convert Statuses to Columns
shipment_pivot = count_shipments_statusdf.pivot(index='id', columns='ShipmentStatus', values='Shipments')
shipment_pivot.tail()

ShipmentStatus,Completed,Executing,Released
id,,,
244354,NaN,NaN,4.0
244355,NaN,NaN,25.0
244356,NaN,NaN,23.0
244358,NaN,NaN,1.0
244362,NaN,NaN,1.0


In [7]:
# Convert NaN to 0
shipment_pivot = shipment_pivot.fillna(0)
shipment_pivot.head()

ShipmentStatus,Completed,Executing,Released
id,,,
244256,0.0,0.0,25.0
244261,0.0,0.0,1.0
244268,0.0,1.0,0.0
244274,0.0,0.0,1.0
244275,0.0,0.0,1.0


In [8]:
# Convert Float to Int
try:
    shipment_pivot['Completed'] = shipment_pivot['Completed'].astype(int)
except:
    print('Column Completed does not exist yet')

try:
    shipment_pivot['Executing'] = shipment_pivot['Executing'].astype(int)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'] = shipment_pivot['Released'].astype(int)
except:
    print('Column Released does not exist yet')

shipment_pivot.head()

ShipmentStatus,Completed,Executing,Released
id,,,
244256,0,0,25
244261,0,0,1
244268,0,1,0
244274,0,0,1
244275,0,0,1


In [9]:
# Convert id index to a column
shipment_pivot.reset_index(inplace=True)
shipment_pivot.head()

ShipmentStatus,id,Completed,Executing,Released
0,244256,0,0,25
1,244261,0,0,1
2,244268,0,1,0
3,244274,0,0,1
4,244275,0,0,1


In [10]:
# Convert zeros to blanks. This also change the data type to String
try:
    shipment_pivot['Completed'].replace([0, ''], '', inplace=True)
except:
    print('Column Completed does not exist yet')
    
try:
    shipment_pivot['Executing'].replace([0, ''], '', inplace=True)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'].replace([0, ''], '', inplace=True)
except:
    print('Column Released does not exist yet')
    
shipment_pivot.head()

ShipmentStatus,id,Completed,Executing,Released
0,244256,,,25
1,244261,,,1
2,244268,,1,
3,244274,,,1
4,244275,,,1


In [11]:
shipment_pivot.dtypes

ShipmentStatus
id            int64
Completed    object
Executing    object
Released     object
dtype: object

In [12]:
# Change columns order
df = df[['Date', 'id', 'Warehouse', 'Customer']]

In [13]:
# Drops duplicates
df = df.drop_duplicates()

In [14]:
df.head(10)

,Date,id,Warehouse,Customer
0,Aug 17 2022 10:37AM,244362,10,Beach Patient Assistance
1,Aug 17 2022 10:22AM,244358,20,"ACI Healthcare USA, Inc."
2,Aug 17 2022 10:21AM,244356,15,"Brookfield Pharmaceuticals, LLC"
25,Aug 17 2022 10:04AM,244355,15,"Alliance Pharma, Inc."
50,Aug 17 2022 10:01AM,244354,10,ISDIN Corporation
54,Aug 17 2022 9:57AM,244353,10,"Methapharm, Inc."
57,Aug 17 2022 9:56AM,244352,10,"Methapharm, Inc."
62,Aug 17 2022 9:56AM,244350,10,"Methapharm, Inc."
71,Aug 17 2022 9:55AM,244351,20,"ACI Healthcare USA, Inc."
91,Aug 17 2022 9:13AM,244345,20,"ACI Healthcare USA, Inc."


In [15]:
# Merge df with pivot table
df2 = df.merge(shipment_pivot, left_on='id', right_on='id')
df2.head(10)

,Date,id,Warehouse,Customer,Completed,Executing,Released
0,Aug 17 2022 10:37AM,244362,10,Beach Patient Assistance,,,1
1,Aug 17 2022 10:22AM,244358,20,"ACI Healthcare USA, Inc.",,,1
2,Aug 17 2022 10:21AM,244356,15,"Brookfield Pharmaceuticals, LLC",,,23
3,Aug 17 2022 10:04AM,244355,15,"Alliance Pharma, Inc.",,,25
4,Aug 17 2022 10:01AM,244354,10,ISDIN Corporation,,,4
5,Aug 17 2022 9:57AM,244353,10,"Methapharm, Inc.",,,3
6,Aug 17 2022 9:56AM,244352,10,"Methapharm, Inc.",,,5
7,Aug 17 2022 9:56AM,244350,10,"Methapharm, Inc.",,,9
8,Aug 17 2022 9:55AM,244351,20,"ACI Healthcare USA, Inc.",,,20
9,Aug 17 2022 9:13AM,244345,20,"ACI Healthcare USA, Inc.",,1,


In [16]:
wave_df = pd.DataFrame()
statuses = list(df2.columns)

# Reorder columns
if ('Released' in statuses) and ('Executing' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing', 'Completed']]
elif ('Released' in statuses) and ('Executing' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing']]
elif ('Released' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Completed']]
elif ('Released' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released']]
else:
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer']]

In [17]:
wave_df.head(10)

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Aug 17 2022 10:37AM,244362,10,Beach Patient Assistance,1,,
1,Aug 17 2022 10:22AM,244358,20,"ACI Healthcare USA, Inc.",1,,
2,Aug 17 2022 10:21AM,244356,15,"Brookfield Pharmaceuticals, LLC",23,,
3,Aug 17 2022 10:04AM,244355,15,"Alliance Pharma, Inc.",25,,
4,Aug 17 2022 10:01AM,244354,10,ISDIN Corporation,4,,
5,Aug 17 2022 9:57AM,244353,10,"Methapharm, Inc.",3,,
6,Aug 17 2022 9:56AM,244352,10,"Methapharm, Inc.",5,,
7,Aug 17 2022 9:56AM,244350,10,"Methapharm, Inc.",9,,
8,Aug 17 2022 9:55AM,244351,20,"ACI Healthcare USA, Inc.",20,,
9,Aug 17 2022 9:13AM,244345,20,"ACI Healthcare USA, Inc.",,1,


In [18]:
# Save Waves DF into json file
wave_df.to_json('data/waves.json')

# Warehouse Pie Chart Data

In [19]:
# Save df2 in df3
warehouse_df = wave_df
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Aug 17 2022 10:37AM,244362,10,Beach Patient Assistance,1,,
1,Aug 17 2022 10:22AM,244358,20,"ACI Healthcare USA, Inc.",1,,
2,Aug 17 2022 10:21AM,244356,15,"Brookfield Pharmaceuticals, LLC",23,,
3,Aug 17 2022 10:04AM,244355,15,"Alliance Pharma, Inc.",25,,
4,Aug 17 2022 10:01AM,244354,10,ISDIN Corporation,4,,


In [20]:
# Conver String to Float
try:
    warehouse_df['Released'] = pd.to_numeric(warehouse_df['Released'],errors='coerce')
except:
    print('Column Released does not exist yet')

try:
    warehouse_df['Executing'] = pd.to_numeric(warehouse_df['Executing'],errors='coerce')
except:
    print('Column Executing does not exist yet')

try:
    warehouse_df['Completed'] = pd.to_numeric(warehouse_df['Completed'],errors='coerce')
except:
    print('Column Completed does not exist yet')

warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Aug 17 2022 10:37AM,244362,10,Beach Patient Assistance,1.0,NaN,NaN
1,Aug 17 2022 10:22AM,244358,20,"ACI Healthcare USA, Inc.",1.0,NaN,NaN
2,Aug 17 2022 10:21AM,244356,15,"Brookfield Pharmaceuticals, LLC",23.0,NaN,NaN
3,Aug 17 2022 10:04AM,244355,15,"Alliance Pharma, Inc.",25.0,NaN,NaN
4,Aug 17 2022 10:01AM,244354,10,ISDIN Corporation,4.0,NaN,NaN


In [21]:
# Conver NaN to Zero
warehouse_df = warehouse_df.fillna(0)
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Aug 17 2022 10:37AM,244362,10,Beach Patient Assistance,1.0,0.0,0.0
1,Aug 17 2022 10:22AM,244358,20,"ACI Healthcare USA, Inc.",1.0,0.0,0.0
2,Aug 17 2022 10:21AM,244356,15,"Brookfield Pharmaceuticals, LLC",23.0,0.0,0.0
3,Aug 17 2022 10:04AM,244355,15,"Alliance Pharma, Inc.",25.0,0.0,0.0
4,Aug 17 2022 10:01AM,244354,10,ISDIN Corporation,4.0,0.0,0.0


In [22]:
# Group by Warehouse
warehouse_df = warehouse_df.groupby(['Warehouse']).sum()
warehouse_df

,id,Released,Executing,Completed
Warehouse,,,,
10,4642295,90.0,1.0,0.0
102,244320,1.0,0.0,0.0
15,488711,48.0,0.0,0.0
16,977112,4.0,0.0,0.0
17,488643,2.0,0.0,0.0
19,244328,10.0,26.0,6.0
20,1710222,52.0,2.0,0.0
21,488636,2.0,0.0,0.0


In [23]:
# Reset Index
warehouse_df.reset_index(inplace=True)
warehouse_df

,Warehouse,id,Released,Executing,Completed
0,10,4642295,90.0,1.0,0.0
1,102,244320,1.0,0.0,0.0
2,15,488711,48.0,0.0,0.0
3,16,977112,4.0,0.0,0.0
4,17,488643,2.0,0.0,0.0
5,19,244328,10.0,26.0,6.0
6,20,1710222,52.0,2.0,0.0
7,21,488636,2.0,0.0,0.0


In [24]:
warehouses = list(warehouse_df.columns)
warehouse_df2 = pd.DataFrame()

if ('Released' in warehouses) and ('Executing' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing', 'Completed']]
elif ('Released' in warehouses) and ('Executing' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing']]
elif ('Released' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Completed']]
elif ('Released' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released']]
else:
    warehouse_df2 = warehouse_df[['Warehouse']]

warehouse_df2

,Warehouse,Released,Executing,Completed
0,10,90.0,1.0,0.0
1,102,1.0,0.0,0.0
2,15,48.0,0.0,0.0
3,16,4.0,0.0,0.0
4,17,2.0,0.0,0.0
5,19,10.0,26.0,6.0
6,20,52.0,2.0,0.0
7,21,2.0,0.0,0.0


In [25]:
warehouse_df2 = pd.melt(warehouse_df2, id_vars=["Warehouse"], 
                  var_name="Status", value_name="Value")
warehouse_df2.head()

,Warehouse,Status,Value
0,10,Released,90.0
1,102,Released,1.0
2,15,Released,48.0
3,16,Released,4.0
4,17,Released,2.0


In [26]:
warehouse_df3 = warehouse_df2.pivot(index ='Status', columns='Warehouse', values='Value')
warehouse_df3

Warehouse,10,102,15,16,17,19,20,21
Status,,,,,,,,
Completed,0.0,0.0,0.0,0.0,0.0,6.0,0.0,0.0
Executing,1.0,0.0,0.0,0.0,0.0,26.0,2.0,0.0
Released,90.0,1.0,48.0,4.0,2.0,10.0,52.0,2.0


In [27]:
warehouse_df4 = warehouse_df3.reset_index()
warehouse_df4

Warehouse,Status,10,102,15,16,17,19,20,21
0,Completed,0.0,0.0,0.0,0.0,0.0,6.0,0.0,0.0
1,Executing,1.0,0.0,0.0,0.0,0.0,26.0,2.0,0.0
2,Released,90.0,1.0,48.0,4.0,2.0,10.0,52.0,2.0


In [28]:
warehouse_df4 = warehouse_df4.rename_axis(None, axis=1)
warehouse_df4

,Status,10,102,15,16,17,19,20,21
0,Completed,0.0,0.0,0.0,0.0,0.0,6.0,0.0,0.0
1,Executing,1.0,0.0,0.0,0.0,0.0,26.0,2.0,0.0
2,Released,90.0,1.0,48.0,4.0,2.0,10.0,52.0,2.0


In [29]:
# Save Warehouse into JSON file
warehouse_df4.to_json('data/warehouse.json')

In [30]:
# Close connection
cnxn.close()